In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.agents import tool

In [3]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
search.name

'search'

In [5]:
search.description

'search(query: str) -> str - Search for weather online'

In [6]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [9]:
from pydantic.v1 import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(..., title="Query", description="The search query")

In [10]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [11]:
search.args

{'query': {'title': 'Query',
  'description': 'The search query',
  'type': 'string'}}

In [12]:
search.run("ads")

'42f'

In [21]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., title="Latitude", description="The latitude of the location")
    longitude: float = Field(..., title="Longitude", description="The longitude of the location")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature from OpenMeteo for a given location."""
    BASE_URL = "https://api.open-meteo.com/v1/forecast"

#52.378056, 9.742549
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forcast_days': 1
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"Failed to fetch data from OpenMeteo: {response.status_code}")
    
    
    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [22]:
get_current_temperature.name

'get_current_temperature'

In [23]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'The latitude of the location',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'The longitude of the location',
  'type': 'number'}}

In [24]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature from OpenMeteo for a given location.'

In [25]:
from langchain.tools.render import format_tool_to_openai_function

In [26]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature from OpenMeteo for a given location.',
 'parameters': {'type': 'object',
  'properties': {'latitude': {'description': 'The latitude of the location',
    'type': 'number'},
   'longitude': {'description': 'The longitude of the location',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [28]:
get_current_temperature({"latitude": 52.378056, "longitude": 9.742549})

C:\Users\ivbeljan\AppData\Local\Temp\ipykernel_34564\40191688.py:30: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 7.1°C'

In [30]:
!pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)


In [31]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [32]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [33]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [34]:
search_wikipedia({"query": "Python (programming language)"})

'Page: Python (programming language)\nSummary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.Python consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.\n\n\n\nPage: 

In [35]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [36]:
spec ="""
{
    "openapi": "3.0.1",
    "info": {
        "title": "MyWebAPIForLLM",
        "version": "1.0"
    },
    "paths": {
        "/WeatherForecast": {
            "get": {
                "tags": [
                    "WeatherForecast"
                ],
                "operationId": "GetWeatherForecast",
                "responses": {
                    "200": {
                        "description": "Success",
                        "content": {
                            "text/plain": {
                                "schema": {
                                    "type": "array",
                                    "items": {
                                        "$ref": "#/components/schemas/WeatherForecast"
                                    }
                                }
                            },
                            "application/json": {
                                "schema": {
                                    "type": "array",
                                    "items": {
                                        "$ref": "#/components/schemas/WeatherForecast"
                                    }
                                }
                            },
                            "text/json": {
                                "schema": {
                                    "type": "array",
                                    "items": {
                                        "$ref": "#/components/schemas/WeatherForecast"
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    },
    "components": {
        "schemas": {
            "DateOnly": {
                "type": "object",
                "properties": {
                    "year": {
                        "type": "integer",
                        "format": "int32"
                    },
                    "month": {
                        "type": "integer",
                        "format": "int32"
                    },
                    "day": {
                        "type": "integer",
                        "format": "int32"
                    },
                    "dayOfWeek": {
                        "$ref": "#/components/schemas/DayOfWeek"
                    },
                    "dayOfYear": {
                        "type": "integer",
                        "format": "int32",
                        "readOnly": true
                    },
                    "dayNumber": {
                        "type": "integer",
                        "format": "int32",
                        "readOnly": true
                    }
                },
                "additionalProperties": false
            },
            "DayOfWeek": {
                "enum": [
                    0,
                    1,
                    2,
                    3,
                    4,
                    5,
                    6
                ],
                "type": "integer",
                "format": "int32"
            },
            "WeatherForecast": {
                "type": "object",
                "properties": {
                    "date": {
                        "$ref": "#/components/schemas/DateOnly"
                    },
                    "temperatureC": {
                        "type": "integer",
                        "format": "int32"
                    },
                    "temperatureF": {
                        "type": "integer",
                        "format": "int32",
                        "readOnly": true
                    },
                    "summary": {
                        "type": "string",
                        "nullable": true
                    }
                },
                "additionalProperties": false
            }
        }
    }
}
"""


In [39]:
specification = OpenAPISpec.from_url("http://localhost:5167/swagger/v1/swagger.json")

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


AttributeError: 'super' object has no attribute 'parse_obj'